In [56]:
import warnings
warnings.filterwarnings("ignore")

In [57]:
from crewai import Agent,Task,Crew,LLM

In [58]:
llm= LLM(
    model="ollama/deepseek-r1:1.5b",
    base_url="http://localhost:11434"
)

In [59]:
import os
from dotenv import load_dotenv
load_dotenv()
serper_api_key =os.getenv("SERPER_API_KEY")

In [60]:
from crewai_tools import SerperDevTool,ScrapeWebsiteTool

In [61]:
search_tool=SerperDevTool()
scrape_tool= ScrapeWebsiteTool()


In [62]:
 # Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llm
)

In [63]:
logistics_manager = Agent(
    role="Logistics Manager",
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool,scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llm
)

In [64]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llm
)

**Creating venue Pydantic Object**

In [65]:
from pydantic import BaseModel


class VenueDetails(BaseModel):
    name:str
    address:str
    capacity:str
    booking_status:str

In [72]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the events"
                    "also the Valid JSON object with venue Detaild",
    human_input=True,# it will ask human feedback at end of the result ,whether we like or not
    output_json=VenueDetails,
    output_file="venue_details.json", 
    # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [73]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True, # making true means that the task can run in parellel with the tasks which come after it
    agent=logistics_manager
)

In [74]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=False,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

**Creating the Crew**

In [75]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True,
    # embedder={
    #     "provider": "ollama",
    #     "config": {"model": "mxbai-embed-large"}  # Local embedding model
    # }
)

In [76]:

event_details = {
    'event_topic': "Wedding ceremany",
    'event_description': "A gathering of  relatives of "
                         "Broom and bride "
                         "to enjoy the wedding party",
    'event_city': "Nohar , Rajasthan",
    'tentative_date': "2025-08-10",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Marriege Hall"
}


In [77]:
result = event_management_crew.kickoff(inputs=event_details)


╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 4a929209-351f-454c-8cb0-104ada5ffea0                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8e76dc2a-ff64-413c-af5a-2921b2ffe491
    Status: Executing Task...
╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: Venue C

In [78]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '364, Rajdhani Bazaar, Nohar',
 'booking_status': 'Available',
 'capacity': '100-200 seats',
 'name': 'Rajdhani Bazaar Nohar'}


In [79]:
from IPython.display import Markdown
Markdown("marketing_report.md")

To promote the wedding ceremony for engagement in at least 500 potential attendees, consider the following marketing strategies:

1. **Eventbrite Website**: Use the search tool to find event planning websites that cater to large events (over 500 attendees) and read their content for services and pricing.

2. **Social Media and Influencers**: Promote Eventbrite on platforms like Instagram, Twitter, and LinkedIn to subtly engage with potential attendees.

3. **Virtual Events**: Attend virtual meetups and join guest speaker sessions to add value and show expertise in your industry.

4. **Networking Opportunities**: Engage by joining local or reputable networking groups to connect with the target audience.

5. **Testimonials and Reviews**: Include testimonials from past attendees to validate claims made by Eventbrite's services, enhancing credibility.

By leveraging these strategies and using the tools available, you can effectively market the wedding ceremony and increase attendee engagement.